## Load Python libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import fasttreeshap
import time

## Pre-process training and testing data

In [ ]:
# source of data: https://archive.ics.uci.edu/ml/datasets/superconductivty+data
data = pd.read_csv("../../data/superconductor_train.csv", engine = "python")
train, test = train_test_split(data, test_size = 0.5, random_state = 0)
label_train = train["critical_temp"]
label_test = test["critical_temp"]
train = train.iloc[:, :-1]
test = test.iloc[:, :-1]
print("Training data has {} rows and {} columns.".format(train.shape[0], train.shape[1])) 
print("Testing data has {} rows and {} columns.".format(test.shape[0], test.shape[1])) 

## Train a random forest model and compute SHAP values

In [ ]:
n_estimators = 100  # number of trees in random forest model
max_depth = 8  # maximum depth of any trees in random forest model

In [ ]:
# train a random forest model
rf_model = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, random_state = 0)
rf_model.fit(train, label_train)
print("Mean squared error on testing set is {:.2f}.".format(mean_squared_error(label_test, rf_model.predict(test))))

In [ ]:
# obtain total number of leaves
shap_explainer = fasttreeshap.TreeExplainer(rf_model)
num_leaves = sum(shap_explainer.model.num_nodes) - sum(sum(shap_explainer.model.children_left > 0))
print("Total number of leaves is {}.".format(num_leaves))

In [ ]:
# estimate memory usage of FastTreeSHAP v2 since FastTreeSHAP v2 has a stricter memory constraint than
# TreeSHAP and FastTreeSHAP v1
max_node = max(shap_explainer.model.num_nodes)
max_leaves = (max_node + 1) // 2
max_combinations = 2**max_depth
memory = max_leaves * max_combinations * 8
if memory < 1024:
    print("Memory usage of FastTreeSHAP v2 is around {}B.".format(memory))
elif memory / 1024 < 1024:
    print("Memory usage of FastTreeSHAP v2 is around {}KB.".format(memory / 1024))
elif memory / 1024**2 < 1024:
    print("Memory usage of FastTreeSHAP v2 is around {}MB.".format(memory / 1024**2))
else:
    print("Memory usage of FastTreeSHAP v2 is around {}GB.".format(memory / 1024**3))

### Compute SHAP values via different versions of TreeSHAP

In [ ]:
num_sample = 1000  # number of samples to be explained

In [ ]:
# compute SHAP values via FastTreeSHAP v0 (i.e., original TreeSHAP)
shap_explainer = fasttreeshap.TreeExplainer(rf_model, algorithm = "v0")
shap_values_v0 = shap_explainer(test.iloc[:num_sample]).values
shap_values_v0.shape

In [ ]:
# compute SHAP values via FastTreeSHAP v1
shap_explainer = fasttreeshap.TreeExplainer(rf_model, algorithm = "v1")
shap_values_v1 = shap_explainer(test.iloc[:num_sample]).values
shap_values_v1.shape

In [ ]:
# justify the correctness of FastTreeSHAP v1
print("Maximum difference of SHAP values between v1 and v0 is {}.".format(
    np.max(abs(np.array(shap_values_v1) - np.array(shap_values_v0)))))

In [ ]:
# compute SHAP values via FastTreeSHAP v2
shap_explainer = fasttreeshap.TreeExplainer(rf_model, algorithm = "v2")
shap_values_v2 = shap_explainer(test.iloc[:num_sample]).values
shap_values_v2.shape

In [ ]:
# justify the correctness of FastTreeSHAP v2
print("Maximum difference of SHAP values between v2 and v0 is {}.".format(
    np.max(abs(np.array(shap_values_v2) - np.array(shap_values_v0)))))

In [ ]:
# compute SHAP values via automatic TreeSHAP algorithm selection
shap_explainer = fasttreeshap.TreeExplainer(rf_model, algorithm = "auto")
shap_values_auto = shap_explainer(test.iloc[:num_sample]).values
shap_values_auto.shape

In [ ]:
# justify the correctness of automatically selected TreeSHAP algorithm
# it turns out that "auto" selects "v2" as the most appropriate TreeSHAP algorithm
print("Maximum difference of SHAP values between auto and v0 is {}.".format(
    np.max(abs(np.array(shap_values_auto) - np.array(shap_values_v0)))))

### Compare running times of different versions of TreeSHAP in computing SHAP values

In [ ]:
# compute SHAP values/SHAP interaction values via TreeSHAP algorithm with version "algorithm_version"
def run_fasttreeshap(model, sample, interactions, algorithm_version, num_round, num_sample, shortcut = False):
    shap_explainer = fasttreeshap.TreeExplainer(model, algorithm = algorithm_version, shortcut = shortcut)
    run_time = np.zeros(num_round)
    for i in range(num_round):
        start = time.time()
        shap_values = shap_explainer(sample.iloc[:num_sample], interactions = interactions).values
        run_time[i] = time.time() - start
        print("Round {} takes {} sec.".format(i + 1, run_time[i]))
    print("Average running time of {} is {} sec (std {} sec){}.".format(
        algorithm_version, np.mean(run_time), np.std(run_time), " (with shortcut)" if shortcut else ""))

In [ ]:
num_sample = 1000  # number of samples to be explained
num_round = 5  # number of rounds to record standard deviation of running time

In [ ]:
# run FastTreeSHAP v0 (i.e., original TreeSHAP) multiple times and record its average running time
run_fasttreeshap(
    model = rf_model, sample = test, interactions = False, algorithm_version = "v0", 
    num_round = num_round, num_sample = num_sample)

In [ ]:
# run FastTreeSHAP v1 multiple times and record its average running time
run_fasttreeshap(
    model = rf_model, sample = test, interactions = False, algorithm_version = "v1", 
    num_round = num_round, num_sample = num_sample)

In [ ]:
# run FastTreeSHAP v2 multiple times and record its average running time
run_fasttreeshap(
    model = rf_model, sample = test, interactions = False, algorithm_version = "v2", 
    num_round = num_round, num_sample = num_sample)

In [ ]:
# run automatically selected TreeSHAP algorithm multiple times and record its average running time
# it turns out that "auto" selects "v2" as the most appropriate TreeSHAP algorithm
run_fasttreeshap(
    model = rf_model, sample = test, interactions = False, algorithm_version = "auto", 
    num_round = num_round, num_sample = num_sample)

### Compute SHAP interaction values via different versions of TreeSHAP

In [ ]:
num_sample = 10  # number of samples to be explained

In [ ]:
# compute SHAP interaction values via FastTreeSHAP v0 (i.e., original TreeSHAP)
shap_explainer = fasttreeshap.TreeExplainer(rf_model, algorithm = "v0")
shap_interaction_values_v0 = shap_explainer(test.iloc[:num_sample], interactions = True).values
shap_interaction_values_v0.shape

In [ ]:
# compute SHAP interaction values via FastTreeSHAP v1
shap_explainer = fasttreeshap.TreeExplainer(rf_model, algorithm = "v1")
shap_interaction_values_v1 = shap_explainer(test.iloc[:num_sample], interactions = True).values
shap_interaction_values_v1.shape

In [ ]:
# justify the correctness of FastTreeSHAP v1
print("Maximum difference of SHAP interaction values between v1 and v0 is {}.".format(
    np.max(abs(np.array(shap_interaction_values_v1) - np.array(shap_interaction_values_v0)))))

In [ ]:
# compute SHAP interaction values via automatic TreeSHAP algorithm selection
# v1 is always preferred to v0 in any use cases, and v2 does not support interactions
shap_explainer = fasttreeshap.TreeExplainer(rf_model, algorithm = "auto")
shap_interaction_values_auto = shap_explainer(test.iloc[:num_sample], interactions = True).values
shap_interaction_values_auto.shape

In [ ]:
# justify the correctness of automatically selected TreeSHAP algorithm
print("Maximum difference of SHAP interaction values between auto and v0 is {}.".format(
    np.max(abs(np.array(shap_interaction_values_auto) - np.array(shap_interaction_values_v0)))))

### Compare running times of different versions of TreeSHAP in computing SHAP interaction values

In [ ]:
num_sample = 10  # number of samples to be explained
num_round = 5  # number of rounds to record standard deviation of running time

In [ ]:
# run FastTreeSHAP v0 (i.e., original TreeSHAP) multiple times and record its average running time
run_fasttreeshap(
    model = rf_model, sample = test, interactions = True, algorithm_version = "v0", 
    num_round = num_round, num_sample = num_sample)

In [ ]:
# run FastTreeSHAP v1 multiple times and record its average running time
run_fasttreeshap(
    model = rf_model, sample = test, interactions = True, algorithm_version = "v1", 
    num_round = num_round, num_sample = num_sample)

In [ ]:
# run automatically selected TreeSHAP algorithm multiple times and record its average running time
# v1 is always preferred to v0 in any use cases, and v2 does not support interactions
run_fasttreeshap(
    model = rf_model, sample = test, interactions = True, algorithm_version = "auto", 
    num_round = num_round, num_sample = num_sample)

## Train an xgboost model and compute SHAP values

In [ ]:
n_estimators = 100  # number of trees in random forest model
max_depth = 8  # maximum depth of any trees in random forest model

In [ ]:
# train an xgboost model
xgb_model = xgb.XGBRegressor(
    max_depth = max_depth, learning_rate = 0.1, n_estimators = n_estimators, n_jobs = 4, 
    subsample = 1, colsample_bytree = 1, colsample_bylevel = 1, reg_alpha = 0, reg_lambda = 1,
    scale_pos_weight = 1, random_state = 0)
xgb_model.fit(train, label_train)
print("Mean squared error on testing set is {:.2f}.".format(mean_squared_error(label_test, xgb_model.predict(test))))

In [ ]:
# obtain total number of leaves
shap_explainer = fasttreeshap.TreeExplainer(xgb_model)
num_leaves = sum(shap_explainer.model.num_nodes) - sum(sum(shap_explainer.model.children_left > 0))
print("Total number of leaves is {}.".format(num_leaves))

In [ ]:
# estimate memory usage of FastTreeSHAP v2 since FastTreeSHAP v2 has a stricter memory constraint than
# TreeSHAP and FastTreeSHAP v1
max_node = max(shap_explainer.model.num_nodes)
max_leaves = (max_node + 1) // 2
max_combinations = 2**max_depth
memory = max_leaves * max_combinations * 8
if memory < 1024:
    print("Memory usage of FastTreeSHAP v2 is around {}B.".format(memory))
elif memory / 1024 < 1024:
    print("Memory usage of FastTreeSHAP v2 is around {}KB.".format(memory / 1024))
elif memory / 1024**2 < 1024:
    print("Memory usage of FastTreeSHAP v2 is around {}MB.".format(memory / 1024**2))
else:
    print("Memory usage of FastTreeSHAP v2 is around {}GB.".format(memory / 1024**3))

### Compute SHAP values via different versions of TreeSHAP

In [ ]:
num_sample = 1000  # number of samples to be explained

In [ ]:
# compute SHAP values via "shortcut" (i.e., original TreeSHAP in xgboost library)
# parallel computing is enabled in "shortcut"
shap_explainer = fasttreeshap.TreeExplainer(xgb_model, algorithm = "v0", shortcut = True)
shap_values_shortcut = shap_explainer(test.iloc[:num_sample]).values
shap_values_shortcut.shape

In [ ]:
# compute SHAP values via FastTreeSHAP v0 (i.e., original TreeSHAP in shap library)
shap_explainer = fasttreeshap.TreeExplainer(xgb_model, algorithm = "v0", shortcut = False)
shap_values_v0 = shap_explainer(test.iloc[:num_sample]).values
shap_values_v0.shape

In [ ]:
# justify the correctness of FastTreeSHAP v0
print("Maximum difference of SHAP values between v0 and shortcut is {}.".format(
    np.max(abs(np.array(shap_values_v0) - np.array(shap_values_shortcut)))))

In [ ]:
# compute SHAP values via FastTreeSHAP v1
shap_explainer = fasttreeshap.TreeExplainer(xgb_model, algorithm = "v1", shortcut = False)
shap_values_v1 = shap_explainer(test.iloc[:num_sample]).values
shap_values_v1.shape

In [ ]:
# justify the correctness of FastTreeSHAP v1
print("Maximum difference of SHAP values between v1 and v0 is {}.".format(
    np.max(abs(np.array(shap_values_v1) - np.array(shap_values_v0)))))

In [ ]:
# compute SHAP values via FastTreeSHAP v2
shap_explainer = fasttreeshap.TreeExplainer(xgb_model, algorithm = "v2", shortcut = False)
shap_values_v2 = shap_explainer(test.iloc[:num_sample]).values
shap_values_v2.shape

In [ ]:
# justify the correctness of FastTreeSHAP v2
print("Maximum difference of SHAP values between v2 and v0 is {}.".format(
    np.max(abs(np.array(shap_values_v2) - np.array(shap_values_v0)))))

In [ ]:
# compute SHAP values via automatic TreeSHAP algorithm selection
shap_explainer = fasttreeshap.TreeExplainer(xgb_model, algorithm = "auto", shortcut = False)
shap_values_auto = shap_explainer(test.iloc[:num_sample]).values
shap_values_auto.shape

In [ ]:
# justify the correctness of automatically selected TreeSHAP algorithm
# it turns out that "auto" selects "v2" as the most appropriate TreeSHAP algorithm
print("Maximum difference of SHAP values between auto and v0 is {}.".format(
    np.max(abs(np.array(shap_values_auto) - np.array(shap_values_v0)))))

### Compare running times of different versions of TreeSHAP in computing SHAP values

In [ ]:
num_sample = 1000  # number of samples to be explained
num_round = 5  # number of rounds to record standard deviation of running time

In [ ]:
# run "shortcut" version of TreeSHAP multiple times and record its average running time
run_fasttreeshap(
    model = xgb_model, sample = test, interactions = False, algorithm_version = "v0", 
    num_round = num_round, num_sample = num_sample, shortcut = True)

In [ ]:
# run FastTreeSHAP v0 (i.e., original TreeSHAP) multiple times and record its average running time
run_fasttreeshap(
    model = xgb_model, sample = test, interactions = False, algorithm_version = "v0", 
    num_round = num_round, num_sample = num_sample, shortcut = False)

In [ ]:
# run FastTreeSHAP v1 multiple times and record its average running time
run_fasttreeshap(
    model = xgb_model, sample = test, interactions = False, algorithm_version = "v1", 
    num_round = num_round, num_sample = num_sample, shortcut = False)

In [ ]:
# run FastTreeSHAP v2 multiple times and record its average running time
run_fasttreeshap(
    model = xgb_model, sample = test, interactions = False, algorithm_version = "v2", 
    num_round = num_round, num_sample = num_sample, shortcut = False)

In [ ]:
# run automatically selected TreeSHAP algorithm multiple times and record its average running time
# it turns out that "auto" selects "v2" as the most appropriate TreeSHAP algorithm
run_fasttreeshap(
    model = xgb_model, sample = test, interactions = False, algorithm_version = "auto", 
    num_round = num_round, num_sample = num_sample, shortcut = False)

### Compute SHAP interaction values via different versions of TreeSHAP

In [ ]:
num_sample = 10  # number of samples to be explained

In [ ]:
# compute SHAP interaction values via "shortcut" (i.e., original TreeSHAP in xgboost library)
# parallel computing is enabled in "shortcut"
shap_explainer = fasttreeshap.TreeExplainer(xgb_model, algorithm = "v0", shortcut = True)
shap_interaction_values_shortcut = shap_explainer(test.iloc[:num_sample], interactions = True).values
shap_interaction_values_shortcut.shape

In [ ]:
# compute SHAP interaction values via FastTreeSHAP v0 (i.e., original TreeSHAP in shap library)
shap_explainer = fasttreeshap.TreeExplainer(xgb_model, algorithm = "v0", shortcut = False)
shap_interaction_values_v0 = shap_explainer(test.iloc[:num_sample], interactions = True).values
shap_interaction_values_v0.shape

In [ ]:
# justify the correctness of FastTreeSHAP v0
print("Maximum difference of SHAP interaction values between v0 and shortcut is {}.".format(
    np.max(abs(np.array(shap_interaction_values_v0) - np.array(shap_interaction_values_shortcut)))))

In [ ]:
# compute SHAP interaction values via FastTreeSHAP v1
shap_explainer = fasttreeshap.TreeExplainer(xgb_model, algorithm = "v1", shortcut = False)
shap_interaction_values_v1 = shap_explainer(test.iloc[:num_sample], interactions = True).values
shap_interaction_values_v1.shape

In [ ]:
# justify the correctness of FastTreeSHAP v1
print("Maximum difference of SHAP interaction values between v1 and v0 is {}.".format(
    np.max(abs(np.array(shap_interaction_values_v1) - np.array(shap_interaction_values_v0)))))

In [ ]:
# compute SHAP interaction values via automatic TreeSHAP algorithm selection
# v1 is always preferred to v0 in any use cases, and v2 does not support interactions
shap_explainer = fasttreeshap.TreeExplainer(xgb_model, algorithm = "auto", shortcut = False)
shap_interaction_values_auto = shap_explainer(test.iloc[:num_sample], interactions = True).values
shap_interaction_values_auto.shape

In [ ]:
# justify the correctness of automatically selected TreeSHAP algorithm
print("Maximum difference of SHAP interaction values between auto and v0 is {}.".format(
    np.max(abs(np.array(shap_interaction_values_auto) - np.array(shap_interaction_values_v0)))))

### Compare running times of different versions of TreeSHAP in computing SHAP interaction values

In [ ]:
num_sample = 10  # number of samples to be explained
num_round = 5  # number of rounds to record standard deviation of running time

In [ ]:
# run "shortcut" version of TreeSHAP multiple times and record its average running time
run_fasttreeshap(
    model = xgb_model, sample = test, interactions = True, algorithm_version = "v0", 
    num_round = num_round, num_sample = num_sample, shortcut = True)

In [ ]:
# run FastTreeSHAP v0 (i.e., original TreeSHAP) multiple times and record its average running time
run_fasttreeshap(
    model = xgb_model, sample = test, interactions = True, algorithm_version = "v0", 
    num_round = num_round, num_sample = num_sample, shortcut = False)

In [ ]:
# run FastTreeSHAP v1 multiple times and record its average running time
run_fasttreeshap(
    model = xgb_model, sample = test, interactions = True, algorithm_version = "v1", 
    num_round = num_round, num_sample = num_sample, shortcut = False)

In [ ]:
# run automatically selected TreeSHAP algorithm multiple times and record its average running time
# v1 is always preferred to v0 in any use cases, and v2 does not support interactions
run_fasttreeshap(
    model = xgb_model, sample = test, interactions = True, algorithm_version = "auto", 
    num_round = num_round, num_sample = num_sample, shortcut = False)